# Get isodistances from API
https://docs.traveltime.com/api/reference/distance-map#results-search_id

In [5]:
import requests
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
import folium
import time

In [2]:
# load park entrances in dataframe
entrances_df = pd.read_json('../get_park_data/total_park_entrances.json')
# load park list with distances in dataframe
distances_df = pd.read_json('../get_park_data/distance_data.json', orient='index')
entrances_df.head()

,name,lat,long
0,Luusbarg,53.570125,9.761084
1,Luusbarg,53.570329,9.761556
2,Luusbarg,53.568481,9.760017
3,Luusbarg,53.568468,9.759341
4,Luusbarg,53.567232,9.759298


In [3]:
# walking distances added to park entrances
park_entrances=entrances_df.merge(distances_df, how='left', on = 'name')
park_entrances.head()

,name,lat,long,walk_dist
0,Luusbarg,53.570125,9.761084,500
1,Luusbarg,53.570329,9.761556,500
2,Luusbarg,53.568481,9.760017,500
3,Luusbarg,53.568468,9.759341,500
4,Luusbarg,53.567232,9.759298,500


In [4]:
park_entrances_500 = park_entrances[park_entrances["walk_dist"] == 500]
park_entrances_1000 = park_entrances[park_entrances["walk_dist"] == 1000]
park_entrances_1500 = park_entrances[park_entrances["walk_dist"] == 1500]
park_entrances_500.head()

,name,lat,long,walk_dist
0,Luusbarg,53.570125,9.761084,500
1,Luusbarg,53.570329,9.761556,500
2,Luusbarg,53.568481,9.760017,500
3,Luusbarg,53.568468,9.759341,500
4,Luusbarg,53.567232,9.759298,500


In [19]:
import requests

headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/geo+json',
    'X-Application-Id': 'ea148590',
    'X-Api-Key': '1527186dff933ad044809a8e222f3661',
}

json_data = {
    'departure_searches': [
        {
            'id': 'driving from Trafalgar Square',
            'departure_time': '2023-11-22T10:00:00Z',
            'travel_distance': 900,
            'coords': {
                'lat': 51.507609,
                'lng': -0.128315,
            },
            'transportation': {
                'type': 'driving',
            },
        },
    ],
    'arrival_searches': [
        {
            'id': 'driving to Trafalgar Square',
            'arrival_time': '2023-11-22T11:00:00Z',
            'travel_distance': 900,
            'coords': {
                'lat': 51.507609,
                'lng': -0.128315,
            },
            'transportation': {
                'type': 'driving',
            },
        },
    ],
}

response = requests.post('https://api.traveltimeapp.com/v4/distance-map', headers=headers, json=json_data)

# Note: json_data will not be serialized by requests
# exactly as it was in the original request.
#data = '{\n  "departure_searches": [\n    {\n      "id": "driving from Trafalgar Square",\n      "departure_time": "2023-11-22T10:00:00Z",\n      "travel_distance": 900,\n      "coords": {\n        "lat": 51.507609,\n        "lng": -0.128315\n      },\n      "transportation": {\n        "type": "driving"\n      }\n    }\n  ],\n  "arrival_searches": [\n    {\n      "id": "driving to Trafalgar Square",\n      "arrival_time": "2023-11-22T11:00:00Z",\n      "travel_distance": 900,\n      "coords": {\n        "lat": 51.507609,\n        "lng": -0.128315\n      },\n      "transportation": {\n        "type": "driving"\n      }\n    }\n  ]\n}'
response = requests.post('https://api.traveltimeapp.com/v4/distance-map', headers=headers, data=data)

In [20]:
print(response)

<Response [200]>


In [22]:
import asyncio
from datetime import datetime

from traveltimepy import Driving, Coordinates, TravelTimeSdk

async def main():
    sdk = TravelTimeSdk("ea148590", "1527186dff933ad044809a8e222f3661")
    
    results = await sdk.time_map_async(
        coordinates=[Coordinates(lat=51.507609, lng=-0.128315), Coordinates(lat=51.517609, lng=-0.138315)],
        arrival_time=datetime.now(),
        transportation=Driving()
    )
    print(results)

asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop